In [1]:
pip install sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install pypyodbc

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pypyodbc

In [2]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import pypyodbc
import pandas as pd

###################################

# declares the variables and defines them for the server connections, along with the table names that are going to be assigned
SERVER_NAME = 'tcp:tecafs.database.windows.net,1433'
DATABASE_NAME = 'TecafsSqlDatabase'
TABLE_NAME = 'clean_data'

###################################

# makes the connection to the database with the connection string; has the driver, server name, database name, id, and password
connection_string = f"""
    DRIVER={{ODBC Driver 18 for SQL Server}};
    SERVER={SERVER_NAME};
    DATABASE={DATABASE_NAME};
    Uid={'tecafs2023'};
    Pwd={'Capstone50'};
"""
###################################

# attempted connection string, didn't work though

#connection_string1 = pypyodbc.connect("Driver={ODBC Driver 18 for SQL Server};Server=tcp:tecafs.database.windows.net,1433;Database=TecafsSqlDatabase;Uid=tecafs2023;Pwd={Capstone50};")

###################################


connection_url = URL.create('mssql+pyodbc', query={'odbc_connect': connection_string})
engine = create_engine(connection_url, module=pypyodbc)